In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.amp
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import ReduceLROnPlateau
import albumentations as A
from albumentations.pytorch import ToTensorV2
import numpy as np
from PIL import Image
from tqdm import tqdm
import json
import os
import random

In [ ]:
class LaneDataset(Dataset):
    def __init__(self, annotation_path, root_dir, transform=None):
        with open(annotation_path, 'r') as f:
            self.annotations = json.load(f)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, idx):
        ann = self.annotations[idx]
        img_rel_path = ann['image'].replace('\\', '/')
        mask_rel_path = ann['mask'].replace('\\', '/')
        img_path = os.path.join(self.root_dir, img_rel_path)
        mask_path = os.path.join(self.root_dir, mask_rel_path)

        try:
            image = np.array(Image.open(img_path).convert("RGB"))
            mask = np.array(Image.open(mask_path).convert("L"))
        except FileNotFoundError as e:
            print(f"⚠️ Warning: Skipping invalid sample. Error: {e}")
            image = np.zeros((590, 1640, 3), dtype=np.uint8)
            mask = np.zeros((590, 1640), dtype=np.uint8)

        if self.transform:
            augmented = self.transform(image=image, mask=mask)
            image = augmented["image"]
            mask = augmented["mask"]

        return image, (mask > 0).float().unsqueeze(0)


In [ ]:
# MODEL ARCHITECTURE (ATTENTION U-NET)
class DoubleConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.double_conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, 3, padding=1, bias=False),
            nn.BatchNorm2d(out_channels), nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, 3, padding=1, bias=False),
            nn.BatchNorm2d(out_channels), nn.ReLU(inplace=True),
        )
    def forward(self, x):
        return self.double_conv(x)

class AttentionGate(nn.Module):
    def __init__(self, F_g, F_l, F_int):
        super(AttentionGate, self).__init__()
        self.W_g = nn.Sequential(
            nn.Conv2d(F_g, F_int, kernel_size=1, bias=True),
            nn.BatchNorm2d(F_int)
        )
        self.W_x = nn.Sequential(
            nn.Conv2d(F_l, F_int, kernel_size=1, bias=True),
            nn.BatchNorm2d(F_int)
        )
        self.psi = nn.Sequential(
            nn.Conv2d(F_int, 1, kernel_size=1, bias=True),
            nn.BatchNorm2d(1), nn.Sigmoid()
        )
        self.relu = nn.ReLU(inplace=True)

    def forward(self, g, x):
        g1 = self.W_g(g)
        x1 = self.W_x(x)
        psi = self.relu(g1 + x1)
        psi = self.psi(psi)
        return x * psi

class AttentionUNET(nn.Module):
    def __init__(self, in_channels=3, out_channels=1, features=[64, 128, 256, 512]):
        super(AttentionUNET, self).__init__()
        self.downs = nn.ModuleList()
        self.ups = nn.ModuleList()
        self.pool = nn.MaxPool2d(2, 2)
        
        for feature in features:
            self.downs.append(DoubleConv(in_channels, feature))
            in_channels = feature
            
        self.bottleneck = DoubleConv(features[-1], features[-1] * 2)
        
        for i, feature in enumerate(reversed(features)):
            self.ups.append(nn.ConvTranspose2d(feature * 2, feature, kernel_size=2, stride=2))
            self.ups.append(AttentionGate(F_g=feature, F_l=feature, F_int=feature // 2))
            self.ups.append(DoubleConv(feature * 2, feature))
            
        self.final_conv = nn.Conv2d(features[0], out_channels, kernel_size=1)

    def forward(self, x):
        skip_connections = []
        for down in self.downs:
            x = down(x)
            skip_connections.append(x)
            x = self.pool(x)
            
        x = self.bottleneck(x)
        skip_connections = skip_connections[::-1]
        
        for i in range(0, len(self.ups), 3):
            x = self.ups[i](x)
            skip_connection = skip_connections[i // 3]
            
            att_out = self.ups[i+1](g=x, x=skip_connection)
            
            if x.shape != att_out.shape:
                x = F.interpolate(x, size=att_out.shape[2:])
                
            x = torch.cat((att_out, x), dim=1)
            x = self.ups[i+2](x)
            
        return self.final_conv(x)

In [ ]:
# LOSS, METRICS, AND TRAINING FUNCTIONS
class DiceLoss(nn.Module):
    def __init__(self, smooth=1e-6): super(DiceLoss, self).__init__(); self.smooth = smooth
    def forward(self, preds, targets):
        preds = torch.sigmoid(preds); intersection = (preds * targets).sum()
        dice = (2. * intersection + self.smooth) / (preds.sum() + targets.sum() + self.smooth)
        return 1 - dice

def loss_fn(preds, targets):
    bce = nn.BCEWithLogitsLoss()(preds, targets); dice = DiceLoss()(preds, targets)
    return bce + dice

def dice_score(preds, targets, threshold=0.5, smooth=1e-8):
    preds = (torch.sigmoid(preds) > threshold).float()
    intersection = (preds * targets).sum(dim=(1, 2, 3))
    union = preds.sum(dim=(1, 2, 3)) + targets.sum(dim=(1, 2, 3))
    dice_per_sample = (2. * intersection + smooth) / (union + smooth)
    return dice_per_sample.mean().item()

def train_one_epoch(loader, model, optimizer, scaler, device):
    loop = tqdm(loader, leave=True, desc="Training")
    total_loss, total_dice = 0, 0
    for data, targets in loop:
        data, targets = data.to(device), targets.to(device)
        with torch.amp.autocast(device_type='cuda'):
            preds = model(data)
            loss = loss_fn(preds, targets)
        optimizer.zero_grad()
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        total_loss += loss.item()
        total_dice += dice_score(preds, targets)
        loop.set_postfix_str(f"Loss={loss.item():.4f}")
    return total_loss / len(loader), total_dice / len(loader)

def evaluate(loader, model, device):
    model.eval()
    total_loss, total_dice = 0, 0
    with torch.no_grad():
        for data, targets in loader:
            data, targets = data.to(device), targets.to(device)
            with torch.amp.autocast(device_type='cuda'):
                preds = model(data)
            loss = loss_fn(preds, targets)
            total_loss += loss.item()
            total_dice += dice_score(preds, targets)
    model.train()
    return total_loss / len(loader), total_dice / len(loader)


In [ ]:
def main():
    IMAGE_HEIGHT, IMAGE_WIDTH = 160, 320
    LEARNING_RATE = 1e-4
    BATCH_SIZE = 32
    NUM_EPOCHS = 50
    NUM_WORKERS = 2; PIN_MEMORY = True
    DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
    SAVE_CHECKPOINT_PATH = "best_model_culane_attention.pth.tar"
    
    culane_root = "/kaggle/input/culane"
    train_annotations = "/kaggle/input/culane/CULane/culane_train_annotations.json"
    val_annotations = "/kaggle/input/culane/CULane/culane_val_annotations.json"
    LOAD_CHECKPOINT_PATH = "/kaggle/input/culane-attention-module-trained-model/culane_attention_module_trained_model/best_model_culane_attention.pth.tar"
    
    train_transform = A.Compose([A.Resize(height=IMAGE_HEIGHT, width=IMAGE_WIDTH), A.HorizontalFlip(p=0.5), A.ColorJitter(p=0.5), A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)), ToTensorV2()])
    val_transform = A.Compose([A.Resize(height=IMAGE_HEIGHT, width=IMAGE_WIDTH), A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)), ToTensorV2()])
    
    train_dataset = LaneDataset(train_annotations, culane_root, transform=train_transform)
    val_dataset = LaneDataset(val_annotations, culane_root, transform=val_transform)
    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS, pin_memory=PIN_MEMORY)
    val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS, pin_memory=PIN_MEMORY)
    
    model = AttentionUNET(in_channels=3, out_channels=1).to(DEVICE)
    optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
    scaler = torch.amp.GradScaler()
    scheduler = ReduceLROnPlateau(optimizer, mode="max", factor=0.5, patience=3, min_lr=1e-6)
    
    best_dice_score, start_epoch = -1.0, 0
    
    if os.path.exists(LOAD_CHECKPOINT_PATH):
        print("🔄 Resuming from checkpoint...")
        checkpoint = torch.load(LOAD_CHECKPOINT_PATH, map_location=DEVICE)
        model.load_state_dict(checkpoint["state_dict"])
        try:
            optimizer.load_state_dict(checkpoint["optimizer"])
        except Exception as e:
            print(f"⚠️ Optimizer state could not be loaded: {e}")
        best_dice_score = checkpoint.get("best_dice", -1.0)
        start_epoch = checkpoint.get("epoch", 0)
        print(f"Resumed from epoch {start_epoch + 1}")
    
    for epoch in range(31, NUM_EPOCHS):
        print(f"\nEpoch [{epoch+1}/{NUM_EPOCHS}]")
        train_loss, train_dice = train_one_epoch(train_loader, model, optimizer, scaler, DEVICE)
        val_loss, val_dice = evaluate(val_loader, model, DEVICE)
        
        print(f"Train: Loss={train_loss:.4f}, Dice={train_dice:.4f}")
        print(f"Val:   Loss={val_loss:.4f}, Dice={val_dice:.4f}")
        print(f"Current LR: {optimizer.param_groups[0]['lr']:.6f}")
        
        if val_dice > best_dice_score + 1e-4:
            best_dice_score = val_dice
            print(f"✅ New best model! Saving... (Val Dice={val_dice:.4f})")
            torch.save({"state_dict": model.state_dict(), "optimizer": optimizer.state_dict(), "best_dice": best_dice_score, "epoch": epoch + 1}, SAVE_CHECKPOINT_PATH)
        
        scheduler.step(val_dice)

In [6]:
if __name__ == "__main__":
    main()

🔄 Resuming from checkpoint...
Resumed from epoch 24

Epoch [32/50]


Training: 100%|██████████| 2778/2778 [30:25<00:00,  1.52it/s, Loss=0.1684]


Train: Loss=0.2449, Dice=0.7832
Val:   Loss=0.6739, Dice=0.5980
Current LR: 0.000100

Epoch [33/50]


Training: 100%|██████████| 2778/2778 [30:22<00:00,  1.52it/s, Loss=0.1733]


Train: Loss=0.2367, Dice=0.7901
Val:   Loss=0.6611, Dice=0.5973
Current LR: 0.000100

Epoch [34/50]


...

Train: Loss=0.1815, Dice=0.8382
Val:   Loss=0.6532, Dice=0.5932
Current LR: 0.000050

Epoch [39/50]


Training: 100%|██████████| 2778/2778 [30:29<00:00,  1.52it/s, Loss=0.1954]


Train: Loss=0.1741, Dice=0.8448
Val:   Loss=0.6651, Dice=0.5906
Current LR: 0.000050

Epoch [40/50]


Training:   9%|▉         | 246/2778 [02:42<27:57,  1.51it/s, Loss=0.1404]


KeyboardInterrupt: 